In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import dask.dataframe as dd
import time
import os
import keras
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout, Flatten
from keras.layers import Conv1D, MaxPooling2D
from keras.optimizers import SGD, Adam
from keras.layers.normalization import BatchNormalization

from DataPreparation.dataset_preparation import get_LANL_dataset
from Utilities.evaluation_utils import save_submission

%matplotlib inline
%load_ext autoreload
%autoreload 2

Using TensorFlow backend.


In [2]:
data_dir = '../Data/LANL_Earthquake_prediction/'
validation_split = 0.2
numpy = True
normalize = True
verbose= True

In [3]:
data, X_train_moments = get_LANL_dataset(data_dir, validation_split, numpy, normalize)
if numpy:
    X_train = data['X_train']
    y_train = data['y_train']
    X_val = data['X_val']
    y_val = data['y_val']
    test_dict = data['test_dict']
    if verbose:
        print('')
        print('----- Dataset Description -----')
        print('X_train size: %d' % len(X_train))
        print('X_val size: %d' % len(X_val))
        print('test_dict size: %d' % len(test_dict))
        print('-------------------------------')
else:
    train_df = data['train_df']
    val_df = data['val_df']
    test_dict = data['test_dict']
    if verbose:
        print('')
        print('----- Dataset Description -----')
        print('train_df:')
        print(train_df.count().compute())
        print('-----------')
        print('val_df:')
        print(val_df.count().compute())
        print('-----------')
        print('test_dict size: %d' % len(test_dict))
        print('-------------------------------')

Dask dataframes loaded.
Preprocessing...
Converting to numpy...
Done.

----- Dataset Description -----
X_train size: 503316384
X_val size: 125829096
test_dict size: 2624
-------------------------------


In [4]:
test_seq_length = len(test_dict[list(test_dict.keys())[0]])
print('Length of test sequences: %d' % test_seq_length)

Length of test sequences: 150000


In [5]:
def build_mlp(lr, weight_decay, momentum):
    model = Sequential()
    model.add(Dense(512, input_dim=test_seq_length, kernel_initializer='he_uniform'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    
    # model.add(Dense(4096, kernel_initializer='he_uniform'))
    # model.add(BatchNormalization())
    # model.add(Activation('relu'))
    
    model.add(Dense(512, kernel_initializer='he_uniform'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    
    model.add(Dense(1, activation=None))
#     sgd = SGD(lr=lr, decay=weight_decay, momentum=momentum, nesterov=True)
    adam = Adam(lr)
    model.compile(loss='mean_absolute_error', optimizer=adam)
    return model

In [6]:
num_epochs = 10
epochs_per_set = 1
N = 5000

def chunks(X, y, n, chunk_len):
    X_patch = []
    y_patch = []
    ind = np.random.choice(len(X)-chunk_len, n)
    for i in range(n):
        X_patch.append(X[ind[i]:ind[i]+chunk_len])
        y_patch.append(y[ind[i]+chunk_len])
    X_patch = np.asarray(X_patch)
    y_patch = np.asarray(y_patch)
    return X_patch, y_patch

momentum = 0.9
weight_decay = 0.00000443
lr = 0.04961
# for _ in range(100):
#     lr = 10 ** np.random.uniform(-5,0)
#     weight_decay = 10 ** np.random.uniform(-8,-4)
#     print('Learning Rate: %.5f' % lr)
#     print('Weight decay: %.8f' % weight_decay)
#     model = build_mlp(lr, weight_decay, momentum)
#     for i in range(num_epochs):
#         X_patch, y_patch = chunks(X_train, y_train, N, test_seq_length)
#         model.fit(X_patch, y_patch, batch_size=32, epochs=epochs_per_set)

In [7]:
lr = 0.02508
weight_decay = 0.000000
model = build_mlp(lr, weight_decay, momentum)

Instructions for updating:
Colocations handled automatically by placer.


In [ ]:
num_epochs = 1000
epochs_per_set = 1
N = 5000

for i in range(num_epochs):
    print(i)
    X_patch, y_patch = chunks(X_train, y_train, N, test_seq_length)
    model.fit(X_patch, y_patch, batch_size=32, epochs=epochs_per_set)

0
Instructions for updating:
Use tf.cast instead.
Epoch 1/1
5000/5000 [==============================] - 15s 3ms/step - loss: 2.9121
1
Epoch 1/1
5000/5000 [==============================] - 11s 2ms/step - loss: 2.6465
2
Epoch 1/1
5000/5000 [==============================] - 11s 2ms/step - loss: 2.6079
3
Epoch 1/1
5000/5000 [==============================] - 11s 2ms/step - loss: 2.6515
4
Epoch 1/1
5000/5000 [==============================] - 11s 2ms/step - loss: 2.5888
5
Epoch 1/1
5000/5000 [==============================] - 11s 2ms/step - loss: 2.5942
6
Epoch 1/1
5000/5000 [==============================] - 11s 2ms/step - loss: 2.6131
7
Epoch 1/1
5000/5000 [==============================] - 11s 2ms/step - loss: 2.5715
8
Epoch 1/1
5000/5000 [==============================] - 11s 2ms/step - loss: 2.5586
9
Epoch 1/1
5000/5000 [==============================] - 11s 2ms/step - loss: 2.5961
10
Epoch 1/1
5000/5000 [==============================] - 11s 2ms/step - loss: 2.5525
11
Epoch 1/1
500

5000/5000 [==============================] - 11s 2ms/step - loss: 2.4780
92
Epoch 1/1
5000/5000 [==============================] - 11s 2ms/step - loss: 2.4293
93
Epoch 1/1
5000/5000 [==============================] - 11s 2ms/step - loss: 2.5278
94
Epoch 1/1
5000/5000 [==============================] - 11s 2ms/step - loss: 2.4681
95
Epoch 1/1
5000/5000 [==============================] - 11s 2ms/step - loss: 2.4566
96
Epoch 1/1
5000/5000 [==============================] - 11s 2ms/step - loss: 2.4612
97
Epoch 1/1
5000/5000 [==============================] - 11s 2ms/step - loss: 2.4518
98
Epoch 1/1
5000/5000 [==============================] - 11s 2ms/step - loss: 2.4590
99
Epoch 1/1
5000/5000 [==============================] - 11s 2ms/step - loss: 2.4668
100
Epoch 1/1
5000/5000 [==============================] - 11s 2ms/step - loss: 2.4561
101
Epoch 1/1
5000/5000 [==============================] - 11s 2ms/step - loss: 2.4838
102
Epoch 1/1
5000/5000 [==============================] - 11s 2m

5000/5000 [==============================] - 11s 2ms/step - loss: 2.4629
186
Epoch 1/1
5000/5000 [==============================] - 11s 2ms/step - loss: 2.4705
187
Epoch 1/1
5000/5000 [==============================] - 11s 2ms/step - loss: 2.4358
188
Epoch 1/1
5000/5000 [==============================] - 11s 2ms/step - loss: 2.4376
189
Epoch 1/1
5000/5000 [==============================] - 11s 2ms/step - loss: 2.4227
190
Epoch 1/1
5000/5000 [==============================] - 11s 2ms/step - loss: 2.4071
191
Epoch 1/1
5000/5000 [==============================] - 11s 2ms/step - loss: 2.4324
192
Epoch 1/1
5000/5000 [==============================] - 11s 2ms/step - loss: 2.4597
193
Epoch 1/1
5000/5000 [==============================] - 11s 2ms/step - loss: 2.4331
194
Epoch 1/1
5000/5000 [==============================] - 11s 2ms/step - loss: 2.4435
195
Epoch 1/1
5000/5000 [==============================] - 11s 2ms/step - loss: 2.4191
196
Epoch 1/1
5000/5000 [==============================] 

5000/5000 [==============================] - 11s 2ms/step - loss: 2.4280
278
Epoch 1/1
5000/5000 [==============================] - 11s 2ms/step - loss: 2.4179
279
Epoch 1/1
5000/5000 [==============================] - 11s 2ms/step - loss: 2.4277: 0s - loss: 2
280
Epoch 1/1
5000/5000 [==============================] - 11s 2ms/step - loss: 2.4154
281
Epoch 1/1
5000/5000 [==============================] - 11s 2ms/step - loss: 2.4523
282
Epoch 1/1
5000/5000 [==============================] - 11s 2ms/step - loss: 2.3830
283
Epoch 1/1
5000/5000 [==============================] - 11s 2ms/step - loss: 2.3768: 0s - los
284
Epoch 1/1
5000/5000 [==============================] - 11s 2ms/step - loss: 2.4303
285
Epoch 1/1
5000/5000 [==============================] - 11s 2ms/step - loss: 2.4094
286
Epoch 1/1
5000/5000 [==============================] - 11s 2ms/step - loss: 2.3937
287
Epoch 1/1
5000/5000 [==============================] - 11s 2ms/step - loss: 2.3921
288
Epoch 1/1
5000/5000 [========

5000/5000 [==============================] - 11s 2ms/step - loss: 2.4022
369
Epoch 1/1
5000/5000 [==============================] - 11s 2ms/step - loss: 2.3924
370
Epoch 1/1
5000/5000 [==============================] - 11s 2ms/step - loss: 2.3854: 0s -
371
Epoch 1/1
5000/5000 [==============================] - 11s 2ms/step - loss: 2.4065
372
Epoch 1/1
5000/5000 [==============================] - 11s 2ms/step - loss: 2.4247
373
Epoch 1/1
5000/5000 [==============================] - 11s 2ms/step - loss: 2.3630
374
Epoch 1/1
5000/5000 [==============================] - 11s 2ms/step - loss: 2.4008: 1s - loss: 2. - ETA: 0s -
375
Epoch 1/1
5000/5000 [==============================] - 11s 2ms/step - loss: 2.4381
376
Epoch 1/1
5000/5000 [==============================] - 11s 2ms/step - loss: 2.4633: 0s 
377
Epoch 1/1
5000/5000 [==============================] - 11s 2ms/step - loss: 2.4111
378
Epoch 1/1
5000/5000 [==============================] - 11s 2ms/step - loss: 2.4158
379
Epoch 1/1
5000/

5000/5000 [==============================] - 11s 2ms/step - loss: 2.4046: 0s - loss: 2. - ETA: 0s - loss: 
460
Epoch 1/1
5000/5000 [==============================] - 11s 2ms/step - loss: 2.3927
461
Epoch 1/1
5000/5000 [==============================] - 11s 2ms/step - loss: 2.3859
462
Epoch 1/1
5000/5000 [==============================] - 11s 2ms/step - loss: 2.4110
463
Epoch 1/1
5000/5000 [==============================] - 11s 2ms/step - loss: 2.4160 ETA: 
464
Epoch 1/1
5000/5000 [==============================] - 11s 2ms/step - loss: 2.3563
465
Epoch 1/1
5000/5000 [==============================] - 11s 2ms/step - loss: 2.3796A: 1s - loss - E
466
Epoch 1/1
5000/5000 [==============================] - 11s 2ms/step - loss: 2.3566: 1s
467
Epoch 1/1
5000/5000 [==============================] - 11s 2ms/step - loss: 2.4045
468
Epoch 1/1
5000/5000 [==============================] - 11s 2ms/step - loss: 2.4182: 0s - loss: 2.419
469
Epoch 1/1
5000/5000 [==============================] - 11s 2ms

5000/5000 [==============================] - 11s 2ms/step - loss: 2.3899
549
Epoch 1/1
5000/5000 [==============================] - 11s 2ms/step - loss: 2.3360
550
Epoch 1/1
5000/5000 [==============================] - 11s 2ms/step - loss: 2.4109
551
Epoch 1/1
5000/5000 [==============================] - 11s 2ms/step - loss: 2.3552: 0s - 
552
Epoch 1/1
5000/5000 [==============================] - 11s 2ms/step - loss: 2.3925
553
Epoch 1/1
5000/5000 [==============================] - 11s 2ms/step - loss: 2.4291
554
Epoch 1/1
5000/5000 [==============================] - 11s 2ms/step - loss: 2.3191
555
Epoch 1/1
5000/5000 [==============================] - 11s 2ms/step - loss: 2.3952
556
Epoch 1/1
5000/5000 [==============================] - 11s 2ms/step - loss: 2.3497: 1s
557
Epoch 1/1
5000/5000 [==============================] - 11s 2ms/step - loss: 2.3664: 2s - loss
558
Epoch 1/1
5000/5000 [==============================] - 11s 2ms/step - loss: 2.4385
559
Epoch 1/1
5000/5000 [==========

In [59]:
X_patch, y_patch = chunks(X_val, y_val, 5000, test_seq_length)
model.evaluate(X_patch, y_patch, batch_size=32)

5000/5000 [==============================] - 10s 2ms/step


3.2052777782440187

In [64]:
test_prediction_dict = {}
seg_id = []
X_test = []
for seg_id_, X_test_ in test_dict.items():
    seg_id.append(seg_id_)
    X_test.append(X_test_)
X_test = np.array(X_test)
y_test = model.predict(X_test)
for k,v in zip(seg_id, y_test):
    test_prediction_dict.update({k:v[0]})

save_submission(test_prediction_dict, 'simple_mlp_submission')
# You advanced 13 places on the leaderboard!

# Your submission scored 2.737, which is an improvement of your previous score of 2.795. Great job!
# Rank 1900 out of 2051

In [ ]:
# model = Sequential()
# model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
# model.add(Conv2D(32, (3, 3), activation='relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.25))

# model.add(Conv2D(64, (3, 3), activation='relu'))
# model.add(Conv2D(64, (3, 3), activation='relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.25))

# model.add(Flatten())
# model.add(Dense(100, activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(10, activation='softmax'))

# sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
# model.compile(loss='categorical_crossentropy', optimizer=sgd)

### Compute Global Average

In [ ]:
global_avg = train_df['time_to_failure'].mean().compute()
print('The global average time (s) until an earthquake hits is %.4f' % global_avg)

### Evaluation

In [ ]:
train_df['AE'] = abs(train_df['time_to_failure'] - global_avg)
train_MAE = train_df['AE'].mean().compute()
print('Train MAE: %.5f' % train_MAE)

In [ ]:
val_df['AE'] = abs(val_df['time_to_failure'] - global_avg)
val_MAE = val_df['AE'].mean().compute()
print('Validation MAE: %.5f' % val_MAE)

### Predictions for Test set

In [ ]:
test_prediction_dict = {}
for seg_id, test_df in test_dict.items():
    test_prediction_dict.update({seg_id:global_avg})

save_submission(test_prediction_dict, 'global_average_submission')

In [ ]:
test_MAE = 2.795 # Got this number from submitting results to Kaggle
print('Global Average Test MAE: %.5f' % test_MAE)
print('This got us to rank 1904 from 2043 people who had submitted results.')